## SI671 Final Project
#### Sizhuang He (sizhuang), Yuyuan Ji (jiyuyuan), Yuqing Zhang (zhyuqing)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

### 1. Data Preprocessing

In [11]:
dataset = pd.read_csv('marketing_campaign.csv', sep = '\t')
dataset

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,13-06-2013,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,10-06-2014,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,25-01-2014,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,24-01-2014,8,428,...,3,0,0,0,0,0,0,3,11,0


In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [13]:
# Identify number of columns by type
numerical_features = [feature for feature in dataset.columns if dataset[feature].dtypes != 'O']
discrete_features = [feature for feature in numerical_features if len(dataset[feature].unique())<25]
continuous_features = [feature for feature in numerical_features if feature not in discrete_features]
categorical_features = [feature for feature in dataset.columns if feature not in numerical_features]
binary_categorical_features = [feature for feature in categorical_features if len(dataset[feature].unique()) <=3]
print("Numerical Features Count {}".format(len(numerical_features)))
print("Discrete features Count {}".format(len(discrete_features)))
print("Continuous features Count {}".format(len(continuous_features)))
print("Categorical features Count {}".format(len(categorical_features)))
print("Binary Categorical features Count {}".format(len(binary_categorical_features)))

Numerical Features Count 26
Discrete features Count 16
Continuous features Count 10
Categorical features Count 3
Binary Categorical features Count 0


In [14]:
print("Categorical features are: ", categorical_features)

Categorical features are:  ['Education', 'Marital_Status', 'Dt_Customer']


In [15]:
# Transform part of the categorical features into dummies
dataset = pd.get_dummies(dataset, columns=['Education', 'Marital_Status']) 

In [17]:
# Check if there is null data
dataset.isnull().sum()

ID                          0
Year_Birth                  0
Income                     24
Kidhome                     0
Teenhome                    0
Dt_Customer                 0
Recency                     0
MntWines                    0
MntFruits                   0
MntMeatProducts             0
MntFishProducts             0
MntSweetProducts            0
MntGoldProds                0
NumDealsPurchases           0
NumWebPurchases             0
NumCatalogPurchases         0
NumStorePurchases           0
NumWebVisitsMonth           0
AcceptedCmp3                0
AcceptedCmp4                0
AcceptedCmp5                0
AcceptedCmp1                0
AcceptedCmp2                0
Complain                    0
Z_CostContact               0
Z_Revenue                   0
Response                    0
Education_2n Cycle          0
Education_Basic             0
Education_Graduation        0
Education_Master            0
Education_PhD               0
Marital_Status_Absurd       0
Marital_St

In [18]:
# Fill null data with mean values
from sklearn.impute import SimpleImputer
mean_imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
dataset.Income = mean_imputer.fit_transform(dataset[["Income"]])

In [19]:
# Categorize age data into several groups
def getAgeGroup(age):
  ageGroup = ''
  if age <= 18:
    ageGroup = 'Teenager'
  elif age >18 and age<= 30:
    ageGroup = 'Bachelors'
  elif age >30 and age<=50:
    ageGroup  = 'Adults'
  elif age >50 and age<=70:
    ageGroup = 'Seniors'
  elif age>=70 and age<100:
    ageGroup = 'Retired'
  else:
    ageGroup = 'Expired'
  return ageGroup

In [20]:
dataset['Age'] = dataset.apply(lambda row: 2023- int(row.Year_Birth), axis = 1)
dataset['AgeGroup'] = dataset.apply(lambda row: getAgeGroup(row.Age), axis = 1)

In [21]:
dataset

,ID,Year_Birth,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,...,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO,Age,AgeGroup
0,5524,1957,58138.0,0,0,04-09-2012,58,635,88,546,...,0,0,0,0,1,0,0,0,66,Seniors
1,2174,1954,46344.0,1,1,08-03-2014,38,11,1,6,...,0,0,0,0,1,0,0,0,69,Seniors
2,4141,1965,71613.0,0,0,21-08-2013,26,426,49,127,...,0,0,0,0,0,1,0,0,58,Seniors
3,6182,1984,26646.0,1,0,10-02-2014,26,11,4,20,...,0,0,0,0,0,1,0,0,39,Adults
4,5324,1981,58293.0,1,0,19-01-2014,94,173,43,118,...,0,0,0,1,0,0,0,0,42,Adults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,61223.0,0,1,13-06-2013,46,709,43,182,...,0,0,0,1,0,0,0,0,56,Seniors
2236,4001,1946,64014.0,2,1,10-06-2014,56,406,0,30,...,0,0,0,0,0,1,0,0,77,Retired
2237,7270,1981,56981.0,0,0,25-01-2014,91,908,48,217,...,0,0,1,0,0,0,0,0,42,Adults
2238,8235,1956,69245.0,0,1,24-01-2014,8,428,30,214,...,0,0,0,0,0,1,0,0,67,Seniors


In [22]:
dataset.to_csv("preprocessed.csv")

### 2. Exploratory Data Analysis